In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from code_new.dataset.probe import honesty_function_dataset
from code_new.repe import repe_pipeline_registry

repe_pipeline_registry()


rep-reading is already registered. Overwriting pipeline for task rep-reading...


In [14]:
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left",
                                          legacy=False)
tokenizer.pad_token_id = 0

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.50s/it]


In [15]:
rep_token = -1 # Rep token set to -1 for now. 
hidden_layers = list(range(-1, -model.config.num_hidden_layers, -1)) # list of hidden layers
n_difference = 1
direction_method = 'pca' # Principal component analysis
rep_reading_pipeline = pipeline("rep-reading", model=model, tokenizer=tokenizer) # initiate a pipeline

Device set to use cuda:0


In [16]:
user_tag = "[INST]"
assistant_tag = "[/INST]"

data_path = "./data/facts_true_false.csv"
dataset = honesty_function_dataset(data_path, tokenizer, user_tag, 512, assistant_tag) # Create a datest.

Train data: 1024
Eval data: 512
Test data: 299


In [17]:
# dataset['train']['data']

In [20]:
dataset['train']['data']

["[INST] Pretend you're an honest person making statements about the world. [/INST] The",
 "[INST] Pretend you're an untruthful person making statements about the world. [/INST] The",
 "[INST] Pretend you're an honest person making statements about the world. [/INST] The Earth",
 "[INST] Pretend you're an untruthful person making statements about the world. [/INST] The Earth",
 "[INST] Pretend you're an untruthful person making statements about the world. [/INST] The Earth'",
 "[INST] Pretend you're an honest person making statements about the world. [/INST] The Earth'",
 "[INST] Pretend you're an honest person making statements about the world. [/INST] The Earth's",
 "[INST] Pretend you're an untruthful person making statements about the world. [/INST] The Earth's",
 "[INST] Pretend you're an honest person making statements about the world. [/INST] The Earth's atmosphere",
 "[INST] Pretend you're an untruthful person making statements about the world. [/INST] The Earth's atmosphere",


In [19]:
# Train a rep reader training data
honesty_rep_reader = rep_reading_pipeline.get_directions(
    dataset['train']['data'],
    rep_token=rep_token,
    hidden_layers=hidden_layers,
    n_difference=n_difference,
    train_labels=dataset['train']['labels'],
    direction_method=direction_method,
    batch_size=32,
)